Please first install the required packages:

In [ ]:
install.packages(c("xtable"))

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



## Introduction

An important question in labour economics is what determines the wage of workers. This is a causal question, but we can begin to investigate it from a predictive perspective.

In the following wage example, $Y$ is the hourly wage of a worker and $X$ is a vector of worker's characteristics, e.g., education, experience, gender. Two main questions here are:

* How can we use job-relevant characteristics, such as education and experience, to best predict wages?

* What is the difference in predicted wages between men and women with the same job-relevant characteristics?

In this lab, we focus on the prediction question first.

## Data


The data set we consider is from the 2015 March Supplement of the U.S. Current Population Survey.  We select white non-hispanic individuals, aged 25 to 64 years, and working more than 35 hours per week for at least 50 weeks of the year. We exclude self-employed workers; individuals living in group quarters; individuals in the military, agricultural or private household sectors;  individuals with inconsistent reports on earnings and employment status; individuals with allocated or missing information in any of the variables used in the analysis; and individuals with hourly wage below $3$.

The variable of interest $Y$ (or $\log(Y)$) is the hourly wage rate constructed as the ratio of the annual earnings to the total number of hours worked, which is constructed in turn as the product of number of weeks worked and the usual number of hours worked per week. In our analysis, we also focus on single (never married) workers.

## Data analysis

**Exercise 1:** Load the data set. To do this upload the file "wage2015_subsample_inference.Rdata" which you can find on ILIAS and run the following code:

In [ ]:
load("/content/wage2015_subsample_inference.rdata")
head(data)

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>
10,9.615385,2.263364,1,0,0,0,1,0,0,0,0,1,7,0.49,0.343,0.2401,3600,11,8370,18
12,48.076923,3.872802,0,0,0,0,1,0,0,0,0,1,31,9.61,29.791,92.3521,3050,10,5070,9
15,11.057692,2.403126,0,0,1,0,0,0,0,0,0,1,18,3.24,5.832,10.4976,6260,19,770,4
18,13.942308,2.634928,1,0,0,0,0,1,0,0,0,1,25,6.25,15.625,39.0625,420,1,6990,12
19,28.846154,3.361977,1,0,0,0,1,0,0,0,0,1,22,4.84,10.648,23.4256,2015,6,9470,22
30,11.730769,2.462215,1,0,0,0,1,0,0,0,0,1,1,0.01,0.001,0.0001,1650,5,7460,14


**Exercise 2:** Let's have a look at the structure of the data. How many observations do we have? What is the mean hourly wage? What are the names of the covariates? Also provide the sample mean of the covariates. What is the share of female workers in our sample?

In [ ]:
str(data)
mean(data$wage)

'data.frame':	5150 obs. of  20 variables:
 $ wage : num  9.62 48.08 11.06 13.94 28.85 ...
 $ lwage: num  2.26 3.87 2.4 2.63 3.36 ...
 $ sex  : num  1 0 0 1 1 1 1 0 1 1 ...
 $ shs  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ hsg  : num  0 0 1 0 0 0 1 1 1 0 ...
 $ scl  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ clg  : num  1 1 0 0 1 1 0 0 0 1 ...
 $ ad   : num  0 0 0 1 0 0 0 0 0 0 ...
 $ mw   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ so   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ we   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ ne   : num  1 1 1 1 1 1 1 1 1 1 ...
 $ exp1 : num  7 31 18 25 22 1 42 37 31 4 ...
 $ exp2 : num  0.49 9.61 3.24 6.25 4.84 ...
 $ exp3 : num  0.343 29.791 5.832 15.625 10.648 ...
 $ exp4 : num  0.24 92.35 10.5 39.06 23.43 ...
 $ occ  : Factor w/ 369 levels "10","20","40",..: 159 136 269 23 99 86 226 232 184 146 ...
 $ occ2 : Factor w/ 22 levels "1","2","3","4",..: 11 10 19 1 6 5 17 17 13 10 ...
 $ ind  : Factor w/ 236 levels "370","380","390",..: 204 117 12 165 231 176 171 135 210 201 ...
 $ ind2 : Factor w/ 

[1] 23.41041

In [ ]:
dim(data)

[1] 5150   20

For the outcome variable *wage* and a subset of the raw regressors, we calculate the empirical mean to get familiar with the data.

In [ ]:
# generate a table of means of variables
library(xtable)
Z_subset <- data[which(colnames(data) %in% c("lwage","sex","shs","hsg","scl","clg","ad","mw","so","we","ne","exp1"))]
table <- matrix(0, 12, 1)
table[1:12,1]   <- as.numeric(lapply(Z_subset,mean))
rownames(table) <- c("Log Wage","Sex","Some High School","High School Graduate","Some College","College Graduate", "Advanced Degree","Midwest","South","West","Northeast","Experience")
colnames(table) <- c("Sample mean")
tab<- xtable(table, digits = 2)
tab

,Sample mean
,<dbl>
Log Wage,2.97078670
Sex,0.44446602
Some High School,0.02330097
High School Graduate,0.24388350
Some College,0.27805825
College Graduate,0.31766990
Advanced Degree,0.13708738
Midwest,0.25961165
South,0.29650485


E.g., the share of female workers in our sample is ~44% ($sex=1$ if female).

**Exercise 3:** Construct tht output variable $\log(Y)$ and the covariate matrix $Z$ which includes the characteristics of workers that are given in the data.

In [ ]:
# construct matrices for estimation from the data
Y <- log(data$wage)
n <- length(Y)
Z <- data[-which(colnames(data) %in% c("wage","lwage"))]
p <- dim(Z)[2]

cat("Number of observations:", n, '\n')
cat( "Number of raw regressors:", p)

Number of observations: 5150 
Number of raw regressors: 18

## Prediction Question

Now, we will construct a prediction rule for the logarithm of hourly wage $\log(Y)$, which depends linearly on job-relevant characteristics $X$ (log-linear model):

\begin{equation}
\log(Y) = \beta'X+ \epsilon.
\end{equation}

Our goals are

* Predict wages using various characteristics of workers.

* Assess the predictive performance of a given model using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample MSE and $R^2$.


We employ two different specifications for prediction:


1. Basic Model:   $X$ consists of a set of raw regressors (e.g. gender, experience, education indicators,  occupation and industry indicators and regional indicators).


2. Flexible Model:  $X$ consists of all raw regressors from the basic model plus a dictionary of transformations (e.g., ${exp}^2$ and ${exp}^3$) and additional two-way interactions of a polynomial in experience with other regressors. An example of a regressor created through a two-way interaction is *experience* times the indicator of having a *college degree*.

Using the **Flexible Model** enables us to approximate the real relationship by a more complex regression model and therefore to reduce the bias. The **Flexible Model** increases the range of potential shapes of the estimated regression function. In general, flexible models often deliver higher prediction accuracy but are harder to interpret.

In R, the models are given by:

In [ ]:
basic <- lwage ~ (sex + exp1 + shs + hsg + scl + clg + mw + so + we + occ2 + ind2) # basic model
flex <- lwage ~ sex + (exp1 + exp2 + exp3 + exp4) * (shs + hsg + scl + clg + mw + so + we + occ2 + ind2) # flexible model

**Exercise 4:** Fit both models to our data by running ordinary least squares (ols). *Hint: lm()*. How many covariates do we have in our basic model and flexible model, respectively?

In [ ]:
# 1. basic model
basic <- lwage ~ (sex + exp1 + shs + hsg + scl + clg + mw + so + we + occ2 + ind2)
regbasic <- lm(basic, data=data) # perform ols using the defined model
regbasic # estimated coefficients
cat( "Number of regressors in the basic model:",length(regbasic$coef), '\n') # number of regressors in the Basic Model


Call:
lm(formula = basic, data = data)

Coefficients:
(Intercept)          sex         exp1          shs          hsg          scl  
   3.722235    -0.072857     0.008568    -0.592798    -0.504337    -0.411994  
        clg           mw           so           we        occ22        occ23  
  -0.182216    -0.027541    -0.034454     0.017249    -0.076472    -0.034678  
      occ24        occ25        occ26        occ27        occ28        occ29  
  -0.096202    -0.187915    -0.414933    -0.045987    -0.377847    -0.215752  
     occ210       occ211       occ212       occ213       occ214       occ215  
  -0.010623    -0.455834    -0.307589    -0.361440    -0.499495    -0.464482  
     occ216       occ217       occ218       occ219       occ220       occ221  
  -0.233715    -0.412588    -0.340418    -0.241480    -0.212628    -0.288413  
     occ222        ind23        ind24        ind25        ind26        ind27  
  -0.422394    -0.116836    -0.244493    -0.273533    -0.249368    -0.139588

Number of regressors in the basic model: 51 


##### Note that the basic model consists of $51$ regressors.

In [ ]:
# 2. flexible model
flex <- lwage ~ sex + (exp1 + exp2 + exp3 + exp4) * (shs + hsg + scl + clg + mw + so + we + occ2 + ind2)
regflex <- lm(flex, data=data)
regflex # estimated coefficients
cat( "Number of regressors in the flexible model:",length(regflex$coef)) # number of regressors in the Flexible Model


Call:
lm(formula = flex, data = data)

Coefficients:
(Intercept)          sex         exp1         exp2         exp3         exp4  
  3.8602606   -0.0695532   -0.0677247    1.6362944   -0.9154735    0.1429357  
        shs          hsg          scl          clg           mw           so  
 -0.1233089   -0.5289024   -0.2920581   -0.0411641    0.1106834    0.0224244  
         we        occ22        occ23        occ24        occ25        occ26  
 -0.0215659    0.1613397    0.2101514    0.0708570   -0.3960076   -0.2310611  
      occ27        occ28        occ29       occ210       occ211       occ212  
  0.3147249   -0.1875417   -0.3390270    0.0209545   -0.6424177   -0.0674774  
     occ213       occ214       occ215       occ216       occ217       occ218  
 -0.2329781    0.2562009   -0.1938585   -0.0551256   -0.4156093   -0.4822168  
     occ219       occ220       occ221       occ222        ind23        ind24  
 -0.2579412   -0.3010203   -0.4271811   -0.8694527   -1.2473654   -0.0948281 

Number of regressors in the flexible model: 246

##### Note that the flexible model consists of $246$ regressors.

#### Evaluating the predictive performance of the basic and flexible models

**Exercise 5:** Now, evaluate the performance of both models based on the $R^2_{sample}$ and the $MSE_{sample}$. Which model performs better?

In [ ]:
# Assess predictive performance
sumbasic <- summary(regbasic)
sumflex <- summary(regflex)

# R-squared
R2.1 <- sumbasic$r.squared
cat("R-squared for the basic model: ", R2.1, "\n")
p1 <- dim(sumbasic$coefficients)[1]

R2.2 <- sumflex$r.squared
cat("R-squared for the flexible model: ", R2.2, "\n")
p2 <- dim(sumflex$coefficients)[1]


# MSE and adjusted MSE
MSE1 <- mean(sumbasic$res^2)
cat("MSE for the basic model: ", MSE1, "\n")


MSE2 <-mean(sumflex$res^2)
cat("MSE for the flexible model: ", MSE2, "\n")

R-squared for the basic model:  0.3100465 
R-squared for the flexible model:  0.3511099 
MSE for the basic model:  0.2244251 
MSE for the flexible model:  0.2110681 


In [ ]:
# Output the table
library(xtable)
table <- matrix(0, 2, 3)
table[1,1:3] <- c(p1,R2.1,MSE1)
table[2,1:3] <- c(p2,R2.2,MSE2)
colnames(table) <- c("p","R^2_{sample}","MSE_{sample}")
rownames(table) <- c("basic reg","flexible reg")
tab <- xtable(table, digits =c(0,0,2,2))
tab

,p,R^2_{sample},MSE_{sample}
,<dbl>,<dbl>,<dbl>
basic reg,51,0.3100465,0.2244251
flexible reg,246,0.3511099,0.2110681


Considering the measures above, the flexible model performs slightly better than the basic model.



Next, we like to apply **data splitting** as a more general procedure to deal with potential overfitting if $p/n$ is not small. We illustrate the approach in the following.

## Data Splitting

**Exercise 6:** Measure the prediction quality of the two models via data splitting:

- Randomly split the data into one training sample and one testing sample. Here use a simple method (stratified splitting is a more sophisticated version of splitting that we might consider).
- Use the training sample to estimate the parameters of the Basic Model and the Flexible Model.
- Use the testing sample for evaluation. Predict the (log) $\mathtt{wage}$  of every observation in the testing sample based on the estimated parameters in the training sample.
- Calculate the Mean Squared Prediction Error $MSE_{test}$ based on the testing sample for both prediction models.

In [ ]:
# splitting the data
n = dim(data)[1]
set.seed(123) # to make the results replicable (we will generate random numbers)
random <- sample(1:n, floor(n*4/5))
# draw (4/5)*n random numbers from 1 to n without replacing them
train <- data[random,] # training sample
test <- data[-random,] # testing sample

In [ ]:
# basic model
# estimating the parameters in the training sample
regbasic <- lm(basic, data=train)

# calculating the out-of-sample MSE
trainregbasic <- predict(regbasic, newdata=test)
y.test <- log(test$wage)
MSE.test1 <- sum((y.test-trainregbasic)^2)/length(y.test)
R2.test1<- 1- MSE.test1/var(y.test)

cat("Test MSE for the basic model: ", MSE.test1, " ")

cat("Test R2 for the basic model: ", R2.test1)

Test MSE for the basic model:  0.2448361  Test R2 for the basic model:  0.252464

In the basic model, the $MSE_{test}$ is quite close to the $MSE_{sample}$.

In [ ]:
# flexible model
# estimating the parameters
options(warn=-1) # ignore warnings
regflex <- lm(flex, data=train)

# calculating the out-of-sample MSE
trainregflex<- predict(regflex, newdata=test)
y.test <- log(test$wage)
MSE.test2 <- sum((y.test-trainregflex)^2)/length(y.test)
R2.test2<- 1- MSE.test2/var(y.test)

cat("Test MSE for the flexible model: ", MSE.test2, " ")

cat("Test R2 for the flexible model: ", R2.test2)

Test MSE for the flexible model:  0.2673409  Test R2 for the flexible model:  0.1837519

In the flexible model too, the discrepancy between the $MSE_{test}$ and the $MSE_{sample}$ is not large.

It is worth noticing that the $MSE_{test}$ varies across different data splits. Hence, it is a good idea to average the out-of-sample MSE over different data splits to get valid results.

Nevertheless, we observe that, based on the out-of-sample $MSE$, the basic model using ols regression performs **about as well (or slightly better)** than the flexible model.


Finally, let us summarize the results:

In [ ]:
# Output the comparison table
table2 <- matrix(0, 2,2)
table2[1,1]   <- MSE.test1
table2[2,1]   <- MSE.test2
table2[1,2]   <- R2.test1
table2[2,2]   <- R2.test2

rownames(table2)<- c("basic reg","flexible reg")
colnames(table2)<- c("MSE_{test}", "R^2_{test}")
tab2 <- xtable(table2, digits =3)
#print(tab2,type="latex") # set type="latex" for printing table in LaTeX
tab2

,MSE_{test},R^2_{test}
,<dbl>,<dbl>
basic reg,0.2448361,0.2524640
flexible reg,0.2673409,0.1837519
